# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [1]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [2]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [3]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [4]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  2.220e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -4.528e-16  0.000e+00  6.939e-18  8.080e-04  1.902e-03]
 [ 0.000e+00  4.123e-16  0.000e+00 -1.634e-17  0.000e+00 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -1.110e-16  1.110e-16  1.110e-16 -5.551e-17]
 [ 0.000e+00  0.000e+00  3.331e-16 -2.220e-16 -5.551e-17  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -1.110e-16 -1.665e-16  0.000e+00]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [5]:
a1 = a.copy()
a1[1, 1] = 3

Resulting matix still has full rank, but the naive LU routine breaks down.

In [6]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in matmul
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in matmul
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [7]:
def checkMinorIsZero(a):
    lenght = a.shape[0]
    for i in range(lenght):
        if np.linalg.det(a[:i,:i])==0:
            return False
    return True

checkMinorIsZero(a)

True

### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [8]:

def rotateWoutNan(a):
    l, u = diy_lu(a1)
    if "nan" not in str(l) and "nan" not in str(u):
        return (l,u)
    l, u = diy_lu(np.rot90(a))
    if "nan" not in str(l) and "nan" not in str(u):
        return (l,u)
    l, u = diy_lu(np.rot90(np.rot90(a)))
    if "nan" not in str(l) and "nan" not in str(u):
        return (l,u)
    l, u = diy_lu(np.rot90(np.rot90(np.rot90(a))))
    if "nan" not in str(l) and "nan" not in str(u):
        return (l,u)
    l, u = diy_lu(np.rot90(np.rot90(np.rot90(np.rot90(a)))))
    if "nan" not in str(l) and "nan" not in str(u):
        return (l,u)

print(rotateWoutNan(a1))

(array([[  1.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ],
       [  1.   ,   1.   ,   0.   ,   0.   ,   0.   ,   0.   ],
       [  1.   ,   2.429,   1.   ,   0.   ,   0.   ,   0.   ],
       [  1.   ,   4.636,   5.166,   1.   ,   0.   ,   0.   ],
       [  1.   ,  17.   , -69.279, 203.527,   1.   ,   0.   ],
       [  1.   ,  17.   , 128.8  , 344.08 ,  -3.024,   1.   ]]), array([[ 3.000e+00,  7.500e-01,  4.286e-01,  3.000e-01,  2.308e-01,
         1.875e-01],
       [ 0.000e+00,  1.324e-01,  8.867e-02,  6.585e-02,  5.225e-02,
         4.327e-02],
       [ 0.000e+00,  0.000e+00,  8.261e-03,  8.820e-03,  8.192e-03,
         7.418e-03],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  1.292e-03,  1.903e-03,
         2.142e-03],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00, -5.650e-02,
        -9.515e-02],
       [ 0.000e+00,  0.000e+00,  2.220e-16,  0.000e+00,  0.000e+00,
         9.683e-02]]))


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in matmul
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in matmul
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app
